In [ ]:
!pip install pytesseract pdfplumber python-docx openpyxl pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.6 MB/s eta 0:00:00


In [ ]:
import os
import logging
import pytesseract
import pdfplumber
import fitz
from PIL import Image
from docx import Document
from openpyxl import load_workbook
from pathlib import Path


logging.basicConfig(
    filename='extraction_log.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

SUPPORTED_EXTENSIONS = ['.pdf', '.docx', '.xlsx']

def extract_text_from_pdf(filepath):
    try:
        with pdfplumber.open(filepath) as pdf:
            text = ''.join(page.extract_text() or '' for page in pdf.pages)
        if not text.strip():
            text = extract_text_from_pdf_with_ocr(filepath)
        return text
    except Exception as e:
        logging.error(f"Failed to extract PDF text: {filepath} - {e}")
        return ""

def extract_text_from_pdf_with_ocr(filepath):
    try:
        text = ''
        doc = fitz.open(filepath)
        for page_num in range(len(doc)):
            pix = doc[page_num].get_pixmap(dpi=300)
            image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            text += pytesseract.image_to_string(image)
        return text
    except Exception as e:
        logging.error(f"OCR failed for PDF: {filepath} - {e}")
        return ""

def extract_text_from_docx(filepath):
    try:
        doc = Document(filepath)
        return '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        logging.error(f"Failed to extract DOCX text: {filepath} - {e}")
        return ""

def extract_text_from_xlsx(filepath):
    try:
        workbook = load_workbook(filepath, data_only=True)
        text = ''
        for sheet in workbook.worksheets:
            for row in sheet.iter_rows(values_only=True):
                line = ' '.join([str(cell) for cell in row if cell is not None])
                text += line + '\n'
        return text
    except Exception as e:
        logging.error(f"Failed to extract XLSX text: {filepath} - {e}")
        return ""

def extract_all_text_from_folder(folder_path='uploads'):
    extracted_data = {}
    folder = Path(folder_path)

    if not folder.exists() or not folder.is_dir():
        logging.error(f"Folder not found: {folder_path}")
        return extracted_data

    for filepath in folder.rglob("*"):
        if filepath.suffix.lower() not in SUPPORTED_EXTENSIONS:
            continue

        try:
            logging.info(f"Processing file: {filepath}")
            if filepath.suffix.lower() == '.pdf':
                text = extract_text_from_pdf(filepath)
            elif filepath.suffix.lower() == '.docx':
                text = extract_text_from_docx(filepath)
            elif filepath.suffix.lower() == '.xlsx':
                text = extract_text_from_xlsx(filepath)
            else:
                continue

            extracted_data[str(filepath)] = text

        except Exception as e:
            logging.error(f"Unhandled error while processing {filepath}: {e}")

    return extracted_data

os.makedirs("extracted_texts", exist_ok=True)


for filepath, content in results.items():
    filename = os.path.basename(filepath)
    base_name = os.path.splitext(filename)[0]
    output_path = os.path.join("extracted_texts", base_name + ".txt")

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(content)

print(" Tous les fichiers extraits ont été sauvegardés dans le dossier 'extracted_texts/'")


 Tous les fichiers extraits ont été sauvegardés dans le dossier 'extracted_texts/'
